In [100]:
import inspect
from typing import List
import textwrap
import os

from neo4j_runway import DataModel, IngestionGenerator, PyIngest, Discovery, LLM, GraphDataModeler

In [101]:
print(DataModel.__init__.__doc__)


        The standard Graph Data Model representation in Neo4j Runway.

        Attributes
        ----------
        nodes : List[Node]
            A list of the nodes in the data model.
        relationships : List[Relationship]
            A list of the relationships in the data model.
        metadata: Optional[Dict[str, Any]]
            Metadata from an import source such as Solutions Workbench, by default None
        use_neo4j_naming_conventions : bool, optional
            Whether to convert labels, relationships and properties to Neo4j naming conventions, by default True
        


In [102]:
[m[1].__doc__ for m in inspect.getmembers(DataModel, predicate=inspect.ismethod) if "BaseModel" not in str(m[1]) and not m[0].startswith("_")]

['\n        Construct a DataModel from an arrows data model JSON file.\n\n        Parameters\n        ----------\n        file_path : str\n            The location and name of the arrows.app JSON file to import.\n\n        Returns\n        -------\n        DataModel\n            An instance of a DataModel.\n        ',
 '\n        Construct a DataModel from a Solutions Workbench data model JSON file.\n\n        Parameters\n        ----------\n        file_path : str\n            The location and name of the Solutions Workbench JSON file to import.\n\n        Returns\n        -------\n        DataModel\n            An instance of a DataModel.\n        ']

In [103]:
# def get_function_docstrings_of_class(class_of_interest) -> List[str]:
#     return [(m[0], m[1].__doc__) for m in inspect.getmembers(class_of_interest, predicate=inspect.isfunction) if "BaseModel" not in str(m[1]) and not m[0].startswith("_")]

In [104]:
def format_docstring(docstring: str) -> str:
    if not docstring: return ""
    docstring = docstring.replace("        ", "    ")
    res = ""
    for line in docstring.split("\n"):
        res+=(textwrap.fill(line, subsequent_indent="        ", width=60) + "\n")
    return res


In [105]:
def get_method_docstrings_of_class(class_of_interest) -> List[str]:
    return [(m[0], format_docstring(m[1].__doc__)) for m in inspect.getmembers(class_of_interest, predicate=inspect.isfunction) if "BaseModel" not in str(m[1]) and (not m[0].startswith("_") or m[0] == "__init__")]

In [106]:
def get_properties_of_class(class_of_interest) -> List[str]:
    ignored_props = {"__fields_set__", "model_extra", "model_fields_set"}
    return [(m[0], format_docstring(m[1].__doc__)) for m in inspect.getmembers(class_of_interest) if "BaseModel" not in str(m[1]) and "property" in str(m[1]) and m[0] not in ignored_props]

In [107]:
get_properties_of_class(DataModel)

[('node_dict',
  '\n    Returns a dictionary of node label to Node.\n\n    Returns\n    -------\n    Dict[str, Node]\n        A dictionary with node label keys and Node values.\n\n'),
 ('node_labels',
  '\n    Returns a list of node labels.\n\n    Returns\n    -------\n    List[str]\n        A list of node labels.\n\n'),
 ('relationship_dict',
  '\n    Returns a dictionary of relationship type to\n        Relationships.\n\n    Returns\n    -------\n    Dict[str, Relationship]\n        A dictionary with relationship type keys and\n        Relationship values.\n\n'),
 ('relationship_types',
  '\n    Returns a list of relationship types.\n\n    Returns\n    -------\n    List[str]\n        A list of relationship types.\n\n')]

In [108]:
# [print(x) for x in get_function_docstrings_of_class(DataModel)]
# print()
# [print(x) for x in get_method_docstrings_of_class(DataModel)]

In [109]:
# print("\n".join(get_method_docstrings_of_class(DataModel)[0][1]))

In [110]:
def get_class_name_as_string(class_of_interest) -> str:
    return str(class_of_interest).split(".")[-1][:-2]

In [111]:
print(get_class_name_as_string(DataModel))

DataModel


In [112]:
def format_content(class_of_interest) -> str:
    class_name_string = get_class_name_as_string(class_of_interest)
    methods_as_strings = get_method_docstrings_of_class(class_of_interest)
    properties_as_strings = get_properties_of_class(class_of_interest)

    content = f"# {class_name_string}" + "\n\n## Class Methods\n\n"

    for m in methods_as_strings:
        content+=f"""
{m[0]}
---
{m[1].strip()}

"""
    if len(properties_as_strings) > 0: content+="\n\n## Class Properties\n\n"
    for p in properties_as_strings:
        content+=f"""
{p[0]}
---
{p[1].strip()}

"""
    return content

In [113]:
print(format_content(DataModel))

# DataModel

## Class Methods


__init__
---
The standard Graph Data Model representation in Neo4j
        Runway.

    Attributes
    ----------
    nodes : List[Node]
        A list of the nodes in the data model.
    relationships : List[Relationship]
        A list of the relationships in the data model.
    metadata: Optional[Dict[str, Any]]
        Metadata from an import source such as Solutions
        Workbench, by default None
    use_neo4j_naming_conventions : bool, optional
        Whether to convert labels, relationships and
        properties to Neo4j naming conventions, by default
        True


apply_neo4j_naming_conventions
---
Apply Neo4j naming conventions to all labels,
        relationships and properties in the data model.

    Returns
    -------
    None


to_arrows
---
Output the data model to arrows compatible JSON file.

    Parameters
    ----------
    file_name : str, optional
        The file name, by default "data-model"
    write_file : bool, optional
 

In [114]:
def write_markdown_file(file_path: str, content: str) -> None:
    base_path = "./docs/_pages/"
    path_parts = file_path.split("/")
    path_only = base_path + "/".join(path_parts[:-1])
    os.makedirs(path_only, exist_ok=True)
    with open(f"{base_path}{file_path}", "w") as f:
                f.write(f"""---
permalink: /{file_path[:-3].replace("_", "-")}/
---
""")
                f.write(content)  

In [115]:
write_markdown_file(file_path="api/data_model.md", content=format_content(DataModel))

In [116]:
CLASS_DIR = [
    {"class": DataModel, "file_path": "api/data-model.md"},
    {"class": IngestionGenerator, "file_path": "api/ingestion_generator.md"},
    {"class": Discovery, "file_path": "api/discovery.md"},
    {"class": GraphDataModeler, "file_path": "api/graph_data_modeler.md"},
    {"class": LLM, "file_path": "api/llm.md"},
]

In [117]:
for m in CLASS_DIR:
    print(m)
    write_markdown_file(file_path=m["file_path"], content=format_content(m["class"]))

{'class': <class 'neo4j_runway.models.core.data_model.DataModel'>, 'file_path': 'api/data-model.md'}
{'class': <class 'neo4j_runway.ingestion.generate_ingest.IngestionGenerator'>, 'file_path': 'api/ingestion_generator.md'}
{'class': <class 'neo4j_runway.discovery.discovery.Discovery'>, 'file_path': 'api/discovery.md'}
{'class': <class 'neo4j_runway.modeler.modeler.GraphDataModeler'>, 'file_path': 'api/graph_data_modeler.md'}
{'class': <class 'neo4j_runway.llm.llm.LLM'>, 'file_path': 'api/llm.md'}
